# Creating Structured Output with LangChain and Claude

This notebook demonstrates a simple approach to generate structured output using LangChain with Claude. We'll focus on the Pydantic method, which is the most straightforward and commonly used approach.

Before we start, create `.env` file and paste there Claude API key:

`CLAUDE_API_KEY=your_api_key`

In [ ]:
import os
from dotenv import load_dotenv
from typing import List
from pydantic import BaseModel, Field

from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, JsonOutputParser

# Load environment variables (particularly ANTHROPIC_API_KEY)
load_dotenv()

## Structured Output Example: Movie Recommendation

Let's create a structured output for a movie recommendation system

In [ ]:
# Step 1: Define your data structure using Pydantic


class MovieRecommendation(BaseModel):
    title: str = Field(description="The title of the movie")
    year: int = Field(description="The year the movie was released")
    director: str = Field(description="The director of the movie")
    genre: List[str] = Field(description="List of genres for the movie")
    plot_summary: str = Field(description="A brief summary of the movie plot")
    rating: float = Field(description="Rating from 0.0 to 10.0")
    reasons_to_watch: List[str] = Field(
        description="Reasons why the user might enjoy this movie"
    )

In [ ]:
# Step 2: Create a parser for this model
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)

In [ ]:
# Step 3: Initialize Claude model
model_version = "claude-3-sonnet-20240229"  # Example model version

model = ChatAnthropic(
    model=model_version,  # You can use other Claude models too
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY"),
    temperature=0,
)

In [ ]:
# Step 4: Create a prompt template that includes the format instructions
template = """
Based on the user's movie preferences, recommend a movie with the following details.
{format_instructions}

User preferences: {preferences}
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
# Step 5: Create the chain
chain = prompt | model | parser

In [ ]:
# Step 6: Run the chain
user_preferences = "I enjoy psychological thrillers with plot twists. Some of my favorite directors are Christopher Nolan and David Fincher."

result = chain.invoke({"preferences": user_preferences})
print(result)

The output will be a structured Python object.

## Method 2: JSON Output Parser

In [ ]:
json_parser = JsonOutputParser()

In [ ]:
restaurant_template = """
Generate a detailed restaurant review for {restaurant_name} in JSON format.
The review should include:
- name: The name of the restaurant
- cuisine: The type of cuisine served
- price_range: Dollar signs ($, $$, $$$, or $$$$)
- rating: A numerical rating from 1-5
- pros: A list of highlights about the restaurant
- cons: A list of drawbacks or areas for improvement
- signature_dishes: A list of must-try dishes
- review_text: A brief written review

{format_instructions}
"""

In [ ]:
restaurant_prompt = ChatPromptTemplate.from_template(
    restaurant_template,
    partial_variables={"format_instructions": json_parser.get_format_instructions()},
)

In [ ]:
restaurant_chain = restaurant_prompt | model | json_parser

In [ ]:
from pprint import pprint

# Test the chain
result = restaurant_chain.invoke({"restaurant_name": "Pasta Paradise"})
pprint(result)

## Explanation

1. We defined a `MovieRecommendation` class using Pydantic and restaurant JSON that specifies the structure we want
2. We created a parser that will convert the LLM output into these structures
3. We set up a prompt template that includes instructions for formatting the output
4. We built a simple chain: prompt → model → parser
5. When invoked, the chain generates a structured object that matches our defined schema

This approach ensures that Claude's responses are formatted consistently and can be easily processed by your application. The Pydantic model handles validation and provides clear type hints.